In [1]:
# pulls from the rotowire

In [2]:
# this imports all modified columns correction and in order 
import requests
import pandas as pd
import json
import glob
from IPython.display import display
from datetime import datetime

# define the template URL for API fetching
rotowire_template_url = "https://www.rotowire.com/betting/nfl/tables/nfl-games.php?week={week}"

# define the columns
columns_to_keep = ['gameID', 'year', 'week', 'gameDate', 'gameDateTime', 'homeAway', 'nickname',
                   'oppNickname', 'draftkings_moneyline', 'draftkings_spread', 'draftkings_ou', 'draftkings_score', 
                   'draftkings_oppScore', 'fanduel_moneyline', 'fanduel_spread', 'fanduel_ou', 'fanduel_score', 'fanduel_oppScore',
                   'betrivers_moneyline', 'betrivers_spread', 'betrivers_ou', 'betrivers_score', 'betrivers_oppScore',
                   'mgm_moneyline', 'mgm_spread', 'mgm_ou', 'mgm_score', 'mgm_oppScore'
                  ]

# fetch JSON data from API and filter columns
def import_rotowire_data(week):
    # Format the URL with the given week
    url = rotowire_template_url.format(week=week)
    
    # Fetch the JSON data from the API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Convert the JSON data to a Python object
        json_data = response.json()
        
        if not json_data:
            return None
        
        # Convert the JSON data to a DataFrame
        df = pd.DataFrame(json_data)
        
        # Add the 'week' column manually
        df['week'] = week
        
        # Add the 'year' column derived from the 'gameDate' column
        df['gameDate'] = pd.to_datetime(df['gameDate'], errors='coerce')  # Convert to datetime
        df['year'] = df['gameDate'].dt.year
        
        # Drop all columns that are not in 'columns_to_keep'
        filtered_df = df[columns_to_keep]

        return filtered_df
    else:
        return None

# load missing weeks from local JSON files
def load_json_files():
    # List to hold data for missing weeks
    missing_weeks_data = []

    # Find all JSON files that follow the pattern for week files
    json_files = glob.glob("rotowire_odds_lines_week_*.json")
    
    for json_file in json_files:
        # Extract the week number from the file name
        week_number = int(json_file.split('_')[-1].replace('.json', ''))
        
        # Read the JSON file
        with open(json_file, 'r') as f:
            json_data = json.load(f)
        
        # Convert the JSON data to a DataFrame
        df = pd.DataFrame(json_data)
        
        # Add the week number and derive the year from the gameDate
        df['week'] = week_number
        df['gameDate'] = pd.to_datetime(df['gameDate'], errors='coerce')
        df['year'] = df['gameDate'].dt.year
        
        # Drop all columns that are not in 'columns_to_keep'
        filtered_df = df[columns_to_keep]
        missing_weeks_data.append(filtered_df)
    
    return missing_weeks_data

In [3]:
# fetch team totals and O/U data for multiple weeks (imports json files)
def teamTotals_overUnder():
    all_weeks_data = []

    # fetch data for each week (you can adjust the range as needed)
    for week in range(1, 18):  # adjust range based on the weeks you want to fetch
        df = import_rotowire_data(week)
        if df is not None:
            all_weeks_data.append(df)

    # load missing weeks from local JSON files and append them
    missing_weeks_data = load_json_files()
    all_weeks_data.extend(missing_weeks_data)

    # concatenate all non-empty DataFrames
    if all_weeks_data:
        full_season_data = pd.concat(all_weeks_data, ignore_index=True)

        # sort by 'year' (descending) and 'week' (ascending)
        full_season_data = full_season_data.sort_values(by=['year', 'week'], ascending=[False, True])

        # display the sorted DataFrame
        display(full_season_data)

        # save the sorted DataFrame to a CSV file
        csv_filename = "teamTotals_overUnder.csv"
        full_season_data.to_csv(csv_filename, index=False)
        print(f"Data successfully written to {csv_filename}")
    else:
        print("No data fetched for any week.")

# Call the function to execute the workflow
# teamTotals_overUnder()

,gameID,year,week,gameDate,gameDateTime,homeAway,nickname,oppNickname,draftkings_moneyline,draftkings_spread,...,betrivers_moneyline,betrivers_spread,betrivers_ou,betrivers_score,betrivers_oppScore,mgm_moneyline,mgm_spread,mgm_ou,mgm_score,mgm_oppScore
474,2656644,2024,1,2024-09-08 13:00:00,Sep 8 1:00pm,away,Steelers,Falcons,154,4.0,...,170,4.0,42.0,19.0,23.0,155,3.5,41.5,19.0,22.5
475,2656644,2024,1,2024-09-08 13:00:00,Sep 8 1:00pm,home,Falcons,Steelers,-185,-4.0,...,-210,-4.0,42.0,23.0,19.0,-190,-3.5,41.5,22.5,19.0
476,2656646,2024,1,2024-09-08 13:00:00,Sep 8 1:00pm,away,Cardinals,Bills,250,6.5,...,240,6.5,46.5,20.0,26.5,240,6.5,46.0,19.8,26.3
477,2656646,2024,1,2024-09-08 13:00:00,Sep 8 1:00pm,home,Bills,Cardinals,-310,-6.5,...,-315,-6.5,46.5,26.5,20.0,-300,-6.5,46.0,26.3,19.8
478,2656652,2024,1,2024-09-08 13:00:00,Sep 8 1:00pm,away,Titans,Bears,160,4.0,...,160,4.0,44.0,20.0,24.0,170,3.5,44.5,20.5,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,2656839,2024,17,2024-12-29 16:25:00,Dec 29 4:25pm,home,Eagles,Cowboys,None,None,...,None,None,None,None,None,None,None,None,None,None
410,2656847,2024,17,2024-12-29 20:20:00,Dec 29 8:20pm,away,Dolphins,Browns,None,None,...,None,None,None,None,None,None,None,None,None,None
411,2656847,2024,17,2024-12-29 20:20:00,Dec 29 8:20pm,home,Browns,Dolphins,None,None,...,None,None,None,None,None,None,None,None,None,None
412,2656850,2024,17,2024-12-30 20:15:00,Dec 30 8:15pm,away,Lions,49ers,None,None,...,None,None,None,None,None,None,None,None,None,None


Data successfully written to teamTotals_overUnder.csv


In [4]:
# fetch team totals and O/U data for the current week only
def get_current_week():
    
    # Logic to compute the current NFL week
    current_date = datetime.now()
    
    # Define the start date of Week 1 in the NFL season (adjust this as needed)
    season_start_date = datetime(2024, 9, 4)
    
    # Calculate the week number based on the difference from the start date
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

def teamTotals_overUnder_current_week():
    current_week_data = []

    # Define the current week (you can adjust this to get the actual current week dynamically)
    current_week = get_current_week() 

    # Fetch data for the current week
    df = import_rotowire_data(current_week)
    if df is not None:
        current_week_data.append(df)

    # Concatenate all non-empty DataFrames
    if current_week_data:
        full_current_week_data = pd.concat(current_week_data, ignore_index=True)

        # Sort by 'year' (descending) and 'week' (ascending)
        full_current_week_data = full_current_week_data.sort_values(by=['year', 'week'], ascending=[False, True])

        # Display the sorted DataFrame
        display(full_current_week_data)

        # Save the sorted DataFrame to a CSV file
        csv_filename = "teamTotals_overUnder_current_week.csv"
        full_current_week_data.to_csv(csv_filename, index=False)
        print(f"Data successfully written to {csv_filename}")
    else:
        print("No data fetched for the current week.")

# Call the new function to fetch data for the current week
teamTotals_overUnder_current_week()

,gameID,year,week,gameDate,gameDateTime,homeAway,nickname,oppNickname,draftkings_moneyline,draftkings_spread,...,betrivers_moneyline,betrivers_spread,betrivers_ou,betrivers_score,betrivers_oppScore,mgm_moneyline,mgm_spread,mgm_ou,mgm_score,mgm_oppScore
0,2656699,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Saints,Falcons,124,2.5,...,125,2.5,41.0,19.3,21.8,130,3.0,41.0,19.0,22.0
1,2656699,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Falcons,Saints,-148,-2.5,...,-150,-2.5,41.0,21.8,19.3,-155,-3.0,41.0,22.0,19.0
2,2656706,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Bengals,Panthers,-218,-4.5,...,-210,-4.5,47.5,26.0,21.5,-225,-4.5,47.5,26.0,21.5
3,2656706,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Panthers,Bengals,180,4.5,...,170,4.5,47.5,21.5,26.0,180,4.5,47.5,21.5,26.0
4,2656709,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Rams,Bears,124,3.0,...,132,3.0,40.5,18.8,21.8,130,3.0,40.5,18.8,21.8
5,2656709,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Bears,Rams,-148,-3.0,...,-159,-3.0,40.5,21.8,18.8,-155,-3.0,40.5,21.8,18.8
6,2656713,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Vikings,Packers,124,2.5,...,125,3.0,44.0,20.5,23.5,125,2.5,44.5,21.0,23.5
7,2656713,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Packers,Vikings,-148,-2.5,...,-152,-3.0,44.0,23.5,20.5,-150,-2.5,44.5,23.5,21.0
8,2656718,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Jaguars,Texans,195,6.0,...,220,6.0,44.5,19.3,25.3,220,6.0,44.5,19.3,25.3
9,2656718,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Texans,Jaguars,-238,-6.0,...,-286,-6.0,44.5,25.3,19.3,-275,-6.0,44.5,25.3,19.3


Data successfully written to teamTotals_overUnder_current_week.csv
